# AI Agent Reference + Tutorial
This notebook will be a reference/cheatsheet moving forward on different tools, strategies etc. when it comes to building out AI agents. This tutorial will primarily focus on utiltizing pure python rather than a particular framework for agentic development

## Basic OpenAI API Call

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import json
client = OpenAI()

response = client.responses.create(
    model="gpt-4o",
    input="Write a one-sentence bedtime story about a unicorn."
)
formatted_response = json.dumps(response.to_dict(), indent=2)
print(formatted_response)

{
  "id": "resp_082c7bb69ddab1db0068e418f0ee4481928afb3b4d4cf8247f",
  "created_at": 1759779056.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-4o-2024-08-06",
  "object": "response",
  "output": [
    {
      "id": "msg_082c7bb69ddab1db0068e418f1e1d88192b8e5d7835d226f2d",
      "content": [
        {
          "annotations": [],
          "text": "Under a shimmering moonlit sky, the gentle unicorn whispered dreams of stardust to the sleeping woodland creatures, filling their night with magic and peace.",
          "type": "output_text",
          "logprobs": []
        }
      ],
      "role": "assistant",
      "status": "completed",
      "type": "message"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [],
  "top_p": 1.0,
  "background": false,
  "max_output_tokens": null,
  "max_tool_calls": null,
  "previous_response_id": null,
  "reasoning": {
    "effort": null,
    "

## Structured Outputs 
You can use pydantic objects in order to have the LLMs response conform to a specific schema. This allows for more meaningful interactions with LLMs where they can create real data objects beyond simply returning text.

In [3]:
from pydantic import BaseModel, Field
from datetime import date

class ToDoItem(BaseModel):
    user: str = Field(..., description="The user who created the task")
    task_name: str = Field(..., description="The name of the task")
    description: str = Field(..., description="A brief description of the task")
    deadline: date = Field(..., description="When the task is due")

# Grab current date to inject into prompts when provided relative dates
today = date.today().isoformat()
# Prompt
content = f"Vihaan needs to complete a book report on Percy Jackson and the Lightning Thief by next Friday. The report should be at least 1000 words long and include a summary of the book, character analysis, and personal reflections on the story."
# OpenAI API call to parse the content into a structured ToDoItem
response = client.responses.parse(
    model="gpt-4o-2024-08-06",
    input=[
        {
            "role": "system",
            "content": "You are a helpful assistant that creates structured to-do items when given a description of one in natural language.  In order to calculate relative dates, note that the current date is {today}"
        },
        {
            "role": "user",
            "content": content,
        }
    ],
    text_format=ToDoItem
)

todo_item = response.output_parsed
print(json.dumps(todo_item.model_dump(), indent=2, default=str))

{
  "user": "Vihaan",
  "task_name": "Complete Book Report on Percy Jackson and the Lightning Thief",
  "description": "Report should be at least 1000 words long and include a summary of the book, character analysis, and personal reflections on the story.",
  "deadline": "2023-10-13"
}


## Testing Structured Outputs with Sets of Literals

In [ ]:
from typing import Literal, List
from pydantic import BaseModel, Field
from enum import StrEnum

class DealFlowFinancingRound(StrEnum):
    Healthcare = "healthcare"
    Finance = "finance"
    Biotech = "biotech"
    SupplyChain = "supply-chain"

class CompanyInfo(BaseModel):
    name: str = Field(..., description="The name of the company")
    # industry: Literal["healthcare", "AI", "finance", "biotech"] = Field(..., description="The industry the company operates in")
    industry: DealFlowFinancingRound = Field(..., description="The industry the company operates in")
    num_employees: int = Field(..., description="The number of employees in the company")
    headquarters: str = Field(..., description="The location of the company's headquarters")
    founded_year: int = Field(..., description="The year the company was founded")

content = f'''From: Daniel Park <daniel@peakvc.com
>
To: Vihaan Vulpala <vihaan@lioncrest.vc
>
Cc: Evan Li <evan@cleargrid.io
>
Subject: Quick intro — Evan Li @ ClearGrid (supply-chain ML)

Vihaan — quick intro to Evan Li, founder of ClearGrid, building ML models that predict supplier-level disruptions (30–60 day horizon). Evan has early revenue ($80k ARR) with two regional distributors and is raising $2M seed. I thought this maps to your portfolio. Evan — Vihaan leads enterprise/data at Lioncrest. Would you be open to a 20–30 min intro? — Daniel '''
response = client.responses.parse(
    model="gpt-4o-2024-08-06",
    input=[
        {
            "role": "system",
            "content": "You are a helpful assistant helps VCs extract data from unstructured text such as emails. conversations, documents etc."
        },
        {
            "role": "user",
            "content": content,
        }
    ],
    text_format=CompanyInfo
)

todo_item = response.output_parsed
print(json.dumps(todo_item.model_dump(), indent=2, default=str))


{
  "name": "ClearGrid",
  "industry": "finance",
  "num_employees": 0,
  "headquarters": "",
  "founded_year": 0
}


# Tool Calling 
Now we're getting into the good stuff. Tools--[according Anthropic at least](https://www.anthropic.com/engineering/building-effective-agents#:~:text=Building%20blocks%2C%20workflows%2C%20and%20agents)--are one of the three fundemental building blocks of constructing AI agents (the other two being retrieval and memory). 

Tools allow agents to take action in the world—they’re how an LLM moves beyond just generating text and actually does something. Whether it’s calling an API, querying a database, sending an email, or triggering a workflow, tools are what make an agent useful, interactive, and capable of solving real problems. 

At it's core tool calling has a few standard steps. Let's walk through through them. Follow along in the code with the same numbers:
1. **Defining the tool:** Write a function for each tool you want the LLM to be able to call + a python dict containing the tool's name, a description, and a schema for arguments that the tool needs in order to be called. 
2. 

In [49]:
import requests

# STEP 1: Define tool

# Need both a function...
def get_weather(lat: int, long: int):
    """
    Get the current weather for a given city.
    """
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={long}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

# ...and a dict describing the tool
# This dict is what the LLM will use to understand how to call the function
tools = [{
        "type": "function",
        "name": "get_weather",
        "description": "Get current temperature for provided coordinates in celsius.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {"type": "number"},
                "longitude": {"type": "number"}
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False
        },
        "strict": True
    }]   

# STEP 2: Call the LLM with the tool
input_messages = [{"role": "user", "content": "Whats the weather like in San Francisco?"}]


response = client.responses.create(
    model="gpt-4.1",
    input=input_messages,
    tools=tools,
)

# STEP 3: Parse the response and process tool call
response_type = response.output[0].type
if response_type == "function_call":
    if response.output[0].name == "get_weather":
        # Extract the arguments for the function call
        args = json.loads(response.output[0].arguments)
        latitude = args["latitude"]
        longitude = args["longitude"]
        
        # Call the function with the extracted arguments
        weather = get_weather(latitude, longitude)
        
        # Print the weather information
        print(f"The current temperature in San Francisco is {weather}°C.")



The current temperature in San Francisco is 13.2°C.


You make think that some of the utility that tool calling provides is redundant with sctructured outputs. However, where tool calling truly stands out is providing optionality to the LLM. Allowing to decide what tools it wants to call when. Here's a more complex example illustrating that:

In [ ]:
# STEP 1: Define the tool
# Here we'll define four possible tools for our to-do list application:
# 1. Add a new task
# 2. Breaking it down into subtasks
# 3. Obtaining resources related to the task

tasks = []

class ToDoItem(BaseModel):
    task_name: str = Field(..., description="The name of the task")
    description: str = Field(..., description="A brief description of the task")
    deadline: date = Field(..., description="When the task is due")
    subtasks: list[ToDoItem] = Field(default_factory=list, description="List of subtasks")

def add_task(item: ToDoItem):
    # Simulate creating a task in a database or API
    print(f"Creating task: {item.task_name} with description: {item.description} and deadline: {item.deadline}")
    tasks.append(item)
    return {"status": "success", "current_task_list": [t.model_dump() for t in tasks]}

def break_down_task(subtask_prompt_list: list[str]):
    pass

tools = {
    "add_task": {
        "name": "add_task",
        "description": "Add a new task to the to-do list.",
        "parameters": ToDoItem.model_json_schema(),
        "function": add_task
    },
    "break_down_task": {
        "name": "break_down_task",
        "description": "Break down a task into subtasks.",
        "parameters": {
            "type": "object",
            "properties": {
                "subtask_prompt_list": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "List of subtasks to be created."
                }
            },
            "required": ["subtask_prompt_list"]
        },
        "function": break_down_task
    }
}


system_prompt = """

"""

def process(user_prompt: str):
    response = client.responses.create(
        model="gpt-4o",
        input=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_prompt
             },
        ],å
        tools=tools,
        tool_choice="auto", 
    )



